Download datasets.

In [ ]:
!gdown https://drive.google.com/file/d/1aQ_mpyoxlQ2OgJ7vXkPk04sogAIpbyNF/view?usp=sharing --fuzzy
!gdown https://drive.google.com/file/d/1-1bTtmmSau_o4MHBqT_FizS0-x_TwH4z/view?usp=sharing --fuzzy
!gdown https://drive.google.com/file/d/1-4T8FSnJK0PZmOuwS295BTg68lA5Q49-/view?usp=sharing --fuzzy
!gdown https://drive.google.com/file/d/1-7gXamdIWTDISiyOu40bd0xCfg1Q4a-O/view?usp=sharing --fuzzy
!gdown https://drive.google.com/file/d/1-8A598Uqj2RV705DaFF5HdM2X12bowFx/view?usp=sharing --fuzzy

Save images contained in the different TFRecord partitions to individual npy files in different folders, as well as the corresponding ground truth values.

In [2]:
test_tfrecords_path = '/content/split_{}.record'
csv_path = '/content/ground_truth_{}.csv'
output_path='/content/test_data{}/'

In [3]:
import tensorflow as tf
import numpy as np

def load_tf_records(filepath):
    filenames = tf.io.gfile.glob(filepath)
    dataset = tf.data.TFRecordDataset(filenames,num_parallel_reads=tf.data.experimental.AUTOTUNE)
    return dataset

def tf_records_file_features_description():
    image_feature_description = {
        
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image': tf.io.FixedLenFeature([],tf.string),
        'label/P': tf.io.FixedLenFeature([], tf.float32),
        'label/K': tf.io.FixedLenFeature([], tf.float32),
        'label/Mg': tf.io.FixedLenFeature([], tf.float32),
        'label/Ph': tf.io.FixedLenFeature([], tf.float32),
    }
    return image_feature_description

def decode_dataset(example_proto):
    features=tf.io.parse_single_example(example_proto, tf_records_file_features_description())

    image=features['image']
    height=features['image/height']
    width=features['image/width']
    image=tf.io.decode_raw(image,tf.int16)
    image=tf.reshape(image,[height,width,150])

    P=features['label/P']
    K=features['label/K']
    Mg=features['label/Mg']
    Ph=features['label/Ph']

    height=features['image/height']
    width=features['image/width']

    label=[P,K,Mg,Ph]

    return image, label, height, width

In [7]:
import pandas as pd
import os

i=0

for k in range(1,6):
    os.mkdir('/content/test_data'+str(k))
    dataset = load_tf_records(test_tfrecords_path.format(k)).map(decode_dataset, num_parallel_calls=tf.data.AUTOTUNE)
    gt = np.array(list(dataset.map(lambda image, label, height, width: label)))
    gt = pd.DataFrame(data=gt, columns=["P", "K", "Mg", "pH"])
    gt.to_csv(csv_path.format(k,k), index_label="sample_index")

    for image, label, height, width in dataset:
        np.save(output_path.format(k) + str(i) + '.npy', image)
        i += 1

    print('Successfully converted split_'+str(k)+'.record')
    i = 0

Successfully converted split_1.record
Successfully converted split_2.record
Successfully converted split_3.record
Successfully converted split_4.record
Successfully converted split_5.record
